In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
num_classes = 1000
n_preds = 10000
input_dir = os.path.join(os.getcwd(), '..')
df_output = pd.read_csv('sample.csv')
df_output = df_output.set_index('Class')
df_merged_input = pd.DataFrame()

In [ ]:
df_vocab = pd.read_csv('../data/3rd_challenge_vocabulary.csv')
vocab_idx2label_dict = dict()
for i, label in enumerate(df_vocab['Index']):
    vocab_idx2label_dict[i+1] = label

In [ ]:
for i in range(1, num_classes+1):
    if i % 100 == 0:
        print(i)
    df_inputs = pd.read_csv(os.path.join(input_dir, '%04d.csv'%(i)))
    df_input = pd.DataFrame()
    for j in range(len(df_inputs)):
        df = pd.DataFrame()
        df['new_seg_label_pred'] = df_inputs['seg_label_pred'][j][1:-1].split(',')
        df['new_seg_prob'] = df_inputs['seg_prob'][j][1:-1].split(',')
        df['key'] = j
        df_input = pd.concat([df_input, df], axis=0)
    df_inputs = df_inputs.join(df_input.set_index('key'))
    df_inputs = df_inputs.drop(columns=['seg_label_pred', 'seg_prob'])
    df_inputs['vid_label_pred'] = df_inputs['vid_label_pred'].map(vocab_idx2label_dict)
    df_inputs['new_seg_label_pred'] = (5*(df_inputs['new_seg_label_pred'].astype(int) - 1))
    df_inputs = df_inputs[df_inputs['new_seg_label_pred'] >= 30]
    df_inputs = df_inputs[df_inputs['new_seg_label_pred'] <= 265]
    df_inputs = df_inputs.sort_values(by=['vid_prob', 'new_seg_prob'], ascending=False).head(n_preds)
    df_inputs = df_inputs.drop(columns=['vid_prob'])
    df_merged_input = pd.concat([df_merged_input, df_inputs], axis=0)

In [ ]:
df_merged_input.rename(columns={'new_seg_label_pred': 'seg_label_pred', 'new_seg_prob': 'seg_prob'}, inplace=True)
df_output = df_output.join(df_merged_input.set_index('vid_label_pred'), on='Class')
df_output

In [ ]:
df_output['Segments'] = df_output[['vid_id', 'seg_label_pred']].apply(lambda x: ':'.join(map(str, x)), axis=1)

In [ ]:
df_output

In [ ]:
df_output = df_output.groupby('Class').agg({'Segments': lambda x: ' '.join(map(str, x))})

In [ ]:
df_output.to_csv("sample_submission.csv")

In [ ]:
df_output